# Main Runner

## Imports

In [ ]:
from objects import *
import pickle
import os
import plotly.express as px

## Generate Experiments

In [ ]:
prices = pd.read_parquet("raw_data/spx_stock_prices.parquet")
treasury_rate_files = ["daily-treasury-rates.csv"] + [f"daily-treasury-rates ({i}).csv" for i in range(1, 25)]
rates_df = [pd.read_csv(f"raw_data/{file}", index_col=0) for file in treasury_rate_files]
rates_df = pd.concat(rates_df)
rates_df.index = pd.to_datetime(rates_df.index)
# sort rates_df by date
rates_df = rates_df.sort_index()
print("Start Generation")
generate_experiments(prices, rates_df, 250, "experiments", lookback=48, error_max=10)

# Read in generated experiments
Only needed if running a single experiment. Otherwise, the experiments are read in from the ``experiments`` folder.

In [ ]:
# Get all files in the experiments folder
experiment_names = os.listdir('experiments')
experiments = []
# read in pickle object
for e in experiment_names:
    with open(f"experiments/{e}", 'rb') as f:
        exp = pickle.load(f)
        experiments.append(exp)

In [ ]:
policy = DirectionalIncentiveTradingPolicy(experiments[0], lambda_=0.5)
simulator = MarketSimulator(experiments[0], policy, verbose=False)
simulator.run()
simulator.total_trades

## Evaluate Policies
Run multiple policies on a set of experiments.

In [ ]:
policies = [DIRECTIONAL_TRADING] + \
           [DIRECTIONAL_INCENTIVE_TRADING(0), DIRECTIONAL_INCENTIVE_TRADING(.25), DIRECTIONAL_INCENTIVE_TRADING(.5), DIRECTIONAL_INCENTIVE_TRADING(.75)] + \
           [NAIVE]
multi_sim = MultiSimRunner('experiments', policies)
multi_sim.get_results("base_results.csv")